In [11]:
import contextlib
import json
import os
import tempfile
from enum import Enum

from termcolor import colored

import tensorflow as tf
from tensorflow.python.tools.optimize_for_inference_lib import optimize_for_inference

from bert_serving.server.bert import modeling
from bert_serving.server.helper import import_tf, set_logger

In [13]:
config = tf.ConfigProto(device_count={'GPU': 0}, allow_soft_placement=True)
model_path = "/home/jugs/Desktop/BERT-Pretrained/uncased_L-12_H-768_A-12/1"

In [16]:
config_fp = os.path.join(model_path, "bert_config.json")
init_checkpoint = os.path.join(model_path, "bert_model.ckpt")

In [18]:
with tf.gfile.GFile(config_fp, 'r') as f:
    bert_config = modeling.BertConfig.from_dict(json.load(f))

In [20]:
input_ids = tf.placeholder(tf.int32, (None, None), 'input_ids')
input_mask = tf.placeholder(tf.int32, (None, None), 'input_mask')
input_type_ids = tf.placeholder(tf.int32, (None, None), 'input_type_ids')

In [21]:
input_tensors = [input_ids, input_mask, input_type_ids]

In [22]:
model = modeling.BertModel(
                config=bert_config,
                is_training=False,
                input_ids=input_ids,
                input_mask=input_mask,
                token_type_ids=input_type_ids,
                use_one_hot_embeddings=False)

W1022 14:49:10.114420 139696150726464 deprecation_wrapper.py:119] From /tmp/kfpapp/bert-as-service/server/bert_serving/server/bert/modeling.py:171: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W1022 14:49:10.116362 139696150726464 deprecation_wrapper.py:119] From /tmp/kfpapp/bert-as-service/server/bert_serving/server/bert/modeling.py:411: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.

W1022 14:49:10.503402 139696150726464 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W1022 14:49:10.538345 139696150726464 deprecation.py:323] From /tmp/kfpapp/bert-as-service/server/bert_s

In [25]:
tvars = tf.trainable_variables()
(assignment_map, initialized_variable_names) = modeling.get_assignment_map_from_checkpoint(tvars, init_checkpoint)
tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

In [27]:
minus_mask = lambda x, m: x - tf.expand_dims(1.0 - m, axis=-1) * 1e30
mul_mask = lambda x, m: x * tf.expand_dims(m, axis=-1)
masked_reduce_max = lambda x, m: tf.reduce_max(minus_mask(x, m), axis=1)
masked_reduce_mean = lambda x, m: tf.reduce_sum(mul_mask(x, m), axis=1) / (
        tf.reduce_sum(m, axis=1, keepdims=True) + 1e-10)

In [42]:
with tf.variable_scope("pooling"):
    if 1 == 1:
        encoder_layer = model.all_encoder_layers[11]
    else:
        all_layers = [model.all_encoder_layers[l] for l in range(3)]
        encoder_layer = tf.concat(all_layers, -1)

In [41]:
model.all_encoder_layers

[<tf.Tensor 'bert/encoder/Reshape_2:0' shape=(?, ?, 768) dtype=float32>,
 <tf.Tensor 'bert/encoder/Reshape_3:0' shape=(?, ?, 768) dtype=float32>,
 <tf.Tensor 'bert/encoder/Reshape_4:0' shape=(?, ?, 768) dtype=float32>,
 <tf.Tensor 'bert/encoder/Reshape_5:0' shape=(?, ?, 768) dtype=float32>,
 <tf.Tensor 'bert/encoder/Reshape_6:0' shape=(?, ?, 768) dtype=float32>,
 <tf.Tensor 'bert/encoder/Reshape_7:0' shape=(?, ?, 768) dtype=float32>,
 <tf.Tensor 'bert/encoder/Reshape_8:0' shape=(?, ?, 768) dtype=float32>,
 <tf.Tensor 'bert/encoder/Reshape_9:0' shape=(?, ?, 768) dtype=float32>,
 <tf.Tensor 'bert/encoder/Reshape_10:0' shape=(?, ?, 768) dtype=float32>,
 <tf.Tensor 'bert/encoder/Reshape_11:0' shape=(?, ?, 768) dtype=float32>,
 <tf.Tensor 'bert/encoder/Reshape_12:0' shape=(?, ?, 768) dtype=float32>,
 <tf.Tensor 'bert/encoder/Reshape_13:0' shape=(?, ?, 768) dtype=float32>]

In [43]:
encoder_layer

<tf.Tensor 'bert/encoder/Reshape_13:0' shape=(?, ?, 768) dtype=float32>

In [44]:
input_mask = tf.cast(input_mask, tf.float32)

In [51]:
pooled = masked_reduce_max(encoder_layer, input_mask)

In [52]:
pooled

<tf.Tensor 'Max_1:0' shape=(?, 768) dtype=float32>

In [53]:
pooled = tf.identity(pooled, 'final_encodes')
output_tensors = [pooled]
tmp_g = tf.get_default_graph().as_graph_def()

In [55]:
output_tensors

[<tf.Tensor 'final_encodes:0' shape=(?, 768) dtype=float32>]

In [56]:
tmp_g

node {
  name: "input_ids"
  op: "Placeholder"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: -1
        }
        dim {
          size: -1
        }
      }
    }
  }
}
node {
  name: "input_mask"
  op: "Placeholder"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: -1
        }
        dim {
          size: -1
        }
      }
    }
  }
}
node {
  name: "input_type_ids"
  op: "Placeholder"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: -1
        }
        dim {
          size: -1
        }
      }
    }
  }
}
node {
  name: "Shape"
  op: "Shape"
  input: "input_ids"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "out_type"
    value {
      ty

In [60]:
def convert_variables_to_constants(sess,
                                   input_graph_def,
                                   output_node_names,
                                   variable_names_whitelist=None,
                                   variable_names_blacklist=None,
                                   use_fp16=False):
    from tensorflow.python.framework.graph_util_impl import extract_sub_graph
    from tensorflow.core.framework import graph_pb2
    from tensorflow.core.framework import node_def_pb2
    from tensorflow.core.framework import attr_value_pb2
    from tensorflow.core.framework import types_pb2
    from tensorflow.python.framework import tensor_util

    def patch_dtype(input_node, field_name, output_node):
        if use_fp16 and (field_name in input_node.attr) and (input_node.attr[field_name].type == types_pb2.DT_FLOAT):
            output_node.attr[field_name].CopyFrom(attr_value_pb2.AttrValue(type=types_pb2.DT_HALF))

    inference_graph = extract_sub_graph(input_graph_def, output_node_names)

    variable_names = []
    variable_dict_names = []
    for node in inference_graph.node:
        if node.op in ["Variable", "VariableV2", "VarHandleOp"]:
            variable_name = node.name
            if ((variable_names_whitelist is not None and
                 variable_name not in variable_names_whitelist) or
                    (variable_names_blacklist is not None and
                     variable_name in variable_names_blacklist)):
                continue
            variable_dict_names.append(variable_name)
            if node.op == "VarHandleOp":
                variable_names.append(variable_name + "/Read/ReadVariableOp:0")
            else:
                variable_names.append(variable_name + ":0")
    if variable_names:
        returned_variables = sess.run(variable_names)
    else:
        returned_variables = []
    found_variables = dict(zip(variable_dict_names, returned_variables))

    output_graph_def = graph_pb2.GraphDef()
    how_many_converted = 0
    for input_node in inference_graph.node:
        output_node = node_def_pb2.NodeDef()
        if input_node.name in found_variables:
            output_node.op = "Const"
            output_node.name = input_node.name
            dtype = input_node.attr["dtype"]
            data = found_variables[input_node.name]

            if use_fp16 and dtype.type == types_pb2.DT_FLOAT:
                output_node.attr["value"].CopyFrom(
                    attr_value_pb2.AttrValue(
                        tensor=tensor_util.make_tensor_proto(data.astype('float16'),
                                                             dtype=types_pb2.DT_HALF,
                                                             shape=data.shape)))
            else:
                output_node.attr["dtype"].CopyFrom(dtype)
                output_node.attr["value"].CopyFrom(attr_value_pb2.AttrValue(
                    tensor=tensor_util.make_tensor_proto(data, dtype=dtype.type,
                                                         shape=data.shape)))
            how_many_converted += 1
        elif input_node.op == "ReadVariableOp" and (input_node.input[0] in found_variables):
            # placeholder nodes
            # print('- %s | %s ' % (input_node.name, input_node.attr["dtype"]))
            output_node.op = "Identity"
            output_node.name = input_node.name
            output_node.input.extend([input_node.input[0]])
            output_node.attr["T"].CopyFrom(input_node.attr["dtype"])
            if "_class" in input_node.attr:
                output_node.attr["_class"].CopyFrom(input_node.attr["_class"])
        else:
            # mostly op nodes
            output_node.CopyFrom(input_node)

        patch_dtype(input_node, 'dtype', output_node)
        patch_dtype(input_node, 'T', output_node)
        patch_dtype(input_node, 'DstT', output_node)
        patch_dtype(input_node, 'SrcT', output_node)
        patch_dtype(input_node, 'Tparams', output_node)

        if use_fp16 and ('value' in output_node.attr) and (
                output_node.attr['value'].tensor.dtype == types_pb2.DT_FLOAT):
            # hard-coded value need to be converted as well
            output_node.attr['value'].CopyFrom(attr_value_pb2.AttrValue(
                tensor=tensor_util.make_tensor_proto(
                    output_node.attr['value'].tensor.float_val[0],
                    dtype=types_pb2.DT_HALF)))

        output_graph_def.node.extend([output_node])

    output_graph_def.library.CopyFrom(inference_graph.library)
    return output_graph_def

In [63]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    dtypes = [n.dtype for n in input_tensors]
    tmp_g = optimize_for_inference(
                tmp_g,
                [n.name[:-2] for n in input_tensors],
                [n.name[:-2] for n in output_tensors],
                [dtype.as_datatype_enum for dtype in dtypes],
                False)
    tmp_g = convert_variables_to_constants(sess, tmp_g, [n.name[:-2] for n in output_tensors],
                                                   use_fp16=True)
#     with tf.gfile.GFile("/tmp/kfpapp/bertmodel.pb", 'wb') as f:
#             f.write(tmp_g.SerializeToString())
    tf.saved_model.simple_save(sess, "")
    
    

In [ ]:
with tf.gfile.GFile("", "rb") as f:
    